In [1]:
'''Import packages for loading data, analysing, and plotting'''

import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xesmf as xe
%matplotlib inline
import cartopy
import cartopy.crs as ccrs
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy.ma as ma
import math
import os
import matplotlib.colors as colors
import scipy
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as FA
import mpl_toolkits.axisartist.grid_finder as GF

In [2]:
#Define PMIP generations and model names
pmip_v4='PMIP4'
pmip_v3='PMIP3'
pmip={}
pmip['PMIP4']=['ACCESS-ESM1-5',
               'AWI-ESM-1-1-LR',
               'CESM2',
               'CNRM-CM6-1',
               'EC-Earth3-LR',
               'FGOALS-f3-L',
               'FGOALS-g3',
               'GISS-E2-1-G',
               'HadGEM3-GC31-LL',
               'HadGEM3-GC31',
               'INM-CM4-8',
               'IPSL-CM6A-LR',
               'MIROC-ES2L',
               'MPI-ESM1-2-LR',
               'MRI-ESM2-0',
               'NESM3',
               'NorESM1-F',
               'NorESM2-LM',
               'UofT-CCSM-4']
pmip['PMIP3']=['BCC-CSM1-1',
               'CCSM4',
               'CNRM-CM5',
               'COSMOS-ASO',
               'CSIRO-Mk3L-1-2',
               'CSIRO-Mk3-6-0',
               'EC-EARTH-2-2',
               'FGOALS-g2',
               'FGOALS-s2',
               'GISS-E2-1-G',
               'GISS-E2-R',
               'HadCM3',
               'HadGEM2-CC',
               'HadGEM2-ES',
               'IPSL-CM5A-LR',
               'KCM1-2-2'
               'MIROC-ESM',
               'MPI-ESM-P',
               'MRI-CGCM3']

In [3]:
#No change needs here

'''Define calculating functions'''

#This function will get all available experiment names
def experimentlist():
    exps=[]
    file_path = "../data" 
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for d in dirnames:
            exps.append(d)
    return exps

#This function will get all available model names in the experiment 
def modellist(experiment_name):
    models=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, filenames in os.walk(file_path):
        for f in filenames:
            mname=f.split("_")[0]
            models.append(mname)
    return models

#This function will get all available filenames in the experiment 
def filenamelist(experiment_name):
    filenames=[]
    file_path = "../data/%s" %(experiment_name)
    for dirpaths, dirnames, files in os.walk(file_path):
        for f in files:
            ff='../data/%s/%s'%(experiment_name,f)
            filenames.append(ff)
    return filenames

#This function will identify models in the ensemble
def identify_ensemble_members(variable_name,experiment_name):
    datadir="../data/%s" %(experiment_name)
    ensemble_members=!../scripts/find_experiment_ensemble_members.bash {experiment_name} {variable_name} {datadir}
    return ensemble_members

#This function will list excat model name
def extract_model_name(filename):
    file_no_path=filename.rpartition("/")
    file_strings=file_no_path[2].partition("_")
    model_name=file_strings[0]
    return model_name

def ensemble_members_dict(variable_name,experiment_name):
    ens_mems=identify_ensemble_members(variable_name,experiment_name)
    ens_mems_dict={extract_model_name(ens_mems[0]):ens_mems[0]}
    for mem in ens_mems[1:]:
        ens_mems_dict[extract_model_name(mem)]=mem
    return ens_mems_dict


experimentlist()

['midHolocene-cal-adj',
 'lig127k',
 'midHolocene',
 'lig127-cal-adj',
 'HadISST',
 'piControl',
 'gpcp',
 'historical',
 '1pctCO2',
 'abrupt4xCO2',
 'midPliocene-eoi400',
 'reanalysis',
 'observations',
 'lgm',
 'lgm-cal-adj']

In [4]:
def ensemble_mean(pmip_v,variable_name,experiment_name,filename):
    modellist=[]
    lats=np.arange(-89.5, 90., 1.0)
    lons=np.arange(-179.5, 180, 1.0)
    grid_1x1= xr.Dataset({'lat': (['lat'],lats),
                         'lon': (['lon'],lons)})
    # Use the bash script to find files with the specified variable
    gcm_dict=ensemble_members_dict(variable_name,experiment_name)
    print(gcm_dict)

    #how many of the models are from this generation?
    n_gcms_of_pmip_v=0
    for gcm in gcm_dict:
        if gcm in pmip[pmip_v]:
            n_gcms_of_pmip_v=n_gcms_of_pmip_v+1

    gcms=np.arange(0,n_gcms_of_pmip_v,1)  
    data=np.full((n_gcms_of_pmip_v,len(lats),len(lons)),np.nan)
    n=0
    for gcm in gcm_dict:
        if gcm in pmip[pmip_v]:
            modellist.append(gcm)
            this_file=xr.open_dataset(gcm_dict.get(gcm),decode_times=False)
            this_var=this_file[variable_name]
            this_regridder=xe.Regridder(this_file,grid_1x1,'bilinear', reuse_weights=False,periodic=True) 
            var_1x1=this_regridder(this_var)
            print(var_1x1)
            this_file.close()
            data[n,:,:]=var_1x1
            n=n+1

    #write out dataset        
    ds=xr.Dataset({variable_name:(["gcms","lat","lon"],data),
                   "modellist":(["gcms"],modellist)},
                 coords={'lat': (['lat'],lats),
                         'lon': (['lon'],lons),
                         'gcms':(['gcms'],gcms)})
    ds.to_netcdf(filename)      
    return modellist

In [ ]:
#midHolocene SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_midHolocene_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_midHolocene_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_midHolocene_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_midHolocene_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_midHolocene_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_midHolocene_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

In [ ]:
#midHolocene precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_midHolocene_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='pr_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/pr_djf/PMIP4_midHolocene_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='pr_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/pr_jja/PMIP4_midHolocene_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='pr_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/pr_ann/PMIP3_midHolocene_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='pr_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/pr_djf/PMIP3_midHolocene_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='pr_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/pr_jja/PMIP3_midHolocene_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='pr_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

In [ ]:
#midHolocene two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_midHolocene_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/tas_djf/PMIP4_midHolocene_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='tas_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/tas_jja/PMIP4_midHolocene_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='midHolocene'
variable_name='tas_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/tas_ann/PMIP3_midHolocene_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='tas_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/tas_djf/PMIP3_midHolocene_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='tas_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/tas_jja/PMIP3_midHolocene_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='midHolocene'
variable_name='tas_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

In [ ]:
#1pctCO2 SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_1pctCO2_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_1pctCO2_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_1pctCO2_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_1pctCO2_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_1pctCO2_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_1pctCO2_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#1pctCO2 precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_1pctCO2_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_1pctCO2_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_1pctCO2_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_1pctCO2_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_1pctCO2_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_1pctCO2_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#1pctCO2 two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_1pctCO2_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_1pctCO2_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_1pctCO2_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='1pctCO2'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_1pctCO2_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_1pctCO2_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_1pctCO2_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='1pctCO2'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [ ]:
#abrupt4xCO2 SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_abrupt4xCO2_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_abrupt4xCO2_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_abrupt4xCO2_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_abrupt4xCO2_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_abrupt4xCO2_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_abrupt4xCO2_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#abrupt4xCO2 precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_abrupt4xCO2_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_abrupt4xCO2_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_abrupt4xCO2_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_abrupt4xCO2_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_abrupt4xCO2_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_abrupt4xCO2_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#abrupt4xCO2 two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_abrupt4xCO2_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_abrupt4xCO2_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_abrupt4xCO2_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='abrupt4xCO2'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_abrupt4xCO2_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_abrupt4xCO2_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_abrupt4xCO2_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='abrupt4xCO2'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [ ]:
#historical SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_historical_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_historical_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_historical_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_historical_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_historical_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_historical_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#historical precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_historical_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_historical_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_historical_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_historical_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_historical_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_historical_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#historical two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_historical_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_historical_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_historical_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_historical_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_historical_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_historical_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [5]:
##historical PMIP3 sst - annual mean, djf and jja***
filename= '../outputs/netcdf/sst_ann/PMIP3_historical_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_historical_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_historical_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)

#historical PMIP3 precipitation - annual mean, djf and jja***
filename= '../outputs/netcdf/pr_ann/PMIP3_historical_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_historical_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_historical_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='historical'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

{'ACCESS-ESM1-5': '../data/historical/ACCESS-ESM1-5_historical.cvdp_data.1850-2014.nc', 'AWI-ESM-1-1-LR': '../data/historical/AWI-ESM-1-1-LR_historical.cvdp_data.1850-2014.nc', 'BCC-CSM1-1': '../data/historical/BCC-CSM1-1_historical.cvdp_data.1850-2012.nc', 'CCSM4': '../data/historical/CCSM4_historical.cvdp_data.1850-2005.nc', 'CESM2': '../data/historical/CESM2_historical.cvdp_data.1850-2014.nc', 'CNRM-CM5': '../data/historical/CNRM-CM5_historical.cvdp_data.1850-2005.nc', 'CNRM-CM6-1': '../data/historical/CNRM-CM6-1_historical.cvdp_data.1850-2014.nc', 'CSIRO-Mk3-6-0': '../data/historical/CSIRO-Mk3-6-0_historical.cvdp_data.1850-2005.nc', 'CSIRO-Mk3L-1-2': '../data/historical/CSIRO-Mk3L-1-2_historical.cvdp_data.1851-2000.nc', 'FGOALS-f3-L': '../data/historical/FGOALS-f3-L_historical.cvdp_data.1850-2014.nc', 'FGOALS-g2': '../data/historical/FGOALS-g2_historical.cvdp_data.1900-2014.nc', 'GISS-E2-1-G': '../data/historical/GISS-E2-1-G_historical.cvdp_data.1850-2014.nc', 'GISS-E2-R': '../data

/Users/milesryan-cummings/opt/anaconda3/envs/xesmf_env/lib/python3.7/site-packages/xesmf/frontend.py:476: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


<xarray.DataArray 'sst_spatialmean_ann' (lat: 180, lon: 360)>
array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       ...,
       [-1.53178609, -1.53211053, -1.53243192, ..., -1.5307706 ,
        -1.53111353, -1.53145372],
       [-1.52483492, -1.52503329, -1.52522763, ..., -1.5242324 ,
        -1.52443603, -1.52463581],
       [-1.5161346 , -1.51620074, -1.51626552, ..., -1.51593374,
        -1.51600162, -1.51606823]])
Coordinates:
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear
<xarray.DataArray 'sst_spatialmean_ann' (lat: 180, lon: 360)>
array([[        nan,         nan,         nan, ...,

['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'FGOALS-g2',
 'GISS-E2-1-G',
 'GISS-E2-R',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [9]:
#historical precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_historical_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_historical_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_historical_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='historical'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)

{'ACCESS-ESM1-5': '../data/historical/ACCESS-ESM1-5_historical.cvdp_data.1850-2014.nc', 'AWI-ESM-1-1-LR': '../data/historical/AWI-ESM-1-1-LR_historical.cvdp_data.1850-2014.nc', 'BCC-CSM1-1': '../data/historical/BCC-CSM1-1_historical.cvdp_data.1850-2012.nc', 'CCSM4': '../data/historical/CCSM4_historical.cvdp_data.1850-2005.nc', 'CESM2': '../data/historical/CESM2_historical.cvdp_data.1850-2014.nc', 'CNRM-CM5': '../data/historical/CNRM-CM5_historical.cvdp_data.1850-2005.nc', 'CNRM-CM6-1': '../data/historical/CNRM-CM6-1_historical.cvdp_data.1850-2014.nc', 'CSIRO-Mk3-6-0': '../data/historical/CSIRO-Mk3-6-0_historical.cvdp_data.1850-2005.nc', 'CSIRO-Mk3L-1-2': '../data/historical/CSIRO-Mk3L-1-2_historical.cvdp_data.1851-2000.nc', 'FGOALS-f3-L': '../data/historical/FGOALS-f3-L_historical.cvdp_data.1850-2014.nc', 'FGOALS-g2': '../data/historical/FGOALS-g2_historical.cvdp_data.1900-2014.nc', 'GISS-E2-1-G': '../data/historical/GISS-E2-1-G_historical.cvdp_data.1850-2014.nc', 'GISS-E2-R': '../data

/Users/milesryan-cummings/opt/anaconda3/envs/xesmf_env/lib/python3.7/site-packages/xesmf/frontend.py:476: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


<xarray.DataArray 'pr_spatialmean_ann' (lat: 180, lon: 360)>
array([[0.20760479, 0.20758342, 0.20756686, ..., 0.20759227, 0.20761016,
        0.2076137 ],
       [0.18578903, 0.18624339, 0.18678424, ..., 0.18467998, 0.18500399,
        0.18537581],
       [0.09980417, 0.10227546, 0.10516667, ..., 0.09439616, 0.09581336,
        0.09764986],
       ...,
       [0.55862219, 0.55754934, 0.55709611, ..., 0.56119581, 0.56005682,
        0.55945809],
       [0.57764007, 0.57704657, 0.57656097, ..., 0.57901646, 0.57846703,
        0.5781136 ],
       [0.58198168, 0.58179195, 0.58159493, ..., 0.58241268, 0.58225141,
        0.58213485]])
Coordinates:
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear
<xarray.DataArray 'pr_spatialmean_ann' (lat: 180, lon: 360)>
array([[0.15492536, 0.15467974, 0.1544264 , ..., 0.15586615, 0.15553719,
        0.155211

In [ ]:
#lgm SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_lgm_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_lgm_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_lgm_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_lgm_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_lgm_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_lgm_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#lgm precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_lgm_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_lgm_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_lgm_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_lgm_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_lgm_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_lgm_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#lgm two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_lgm_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_lgm_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_lgm_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='lgm'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_lgm_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_lgm_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_lgm_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='lgm'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [ ]:
#lig127k SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_lig127k_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_lig127k_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_lig127k_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_lig127k_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_lig127k_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_lig127k_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#lig127k precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_lig127k_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_lig127k_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_lig127k_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_lig127k_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_lig127k_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_lig127k_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#lig127k two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_lig127k_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_lig127k_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_lig127k_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='lig127k'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_lig127k_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_lig127k_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_lig127k_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='lig127k'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [ ]:
#midPliocene-eoi400 SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_midPliocene-eoi400_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_midPliocene-eoi400_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_midPliocene-eoi400_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_midPliocene-eoi400_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_midPliocene-eoi400_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_midPliocene-eoi400_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#midPliocene-eoi400 precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_midPliocene-eoi400_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_midPliocene-eoi400_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_midPliocene-eoi400_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_midPliocene-eoi400_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_midPliocene-eoi400_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_midPliocene-eoi400_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#midPliocene-eoi400 two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_midPliocene-eoi400_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_midPliocene-eoi400_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_midPliocene-eoi400_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='midPliocene-eoi400'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_midPliocene-eoi400_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_midPliocene-eoi400_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_midPliocene-eoi400_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='midPliocene-eoi400'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [ ]:
#observations SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_observations_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_observations_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_observations_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_observations_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_observations_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_observations_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#observations precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_observations_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_observations_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_observations_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_observations_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_observations_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_observations_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#observations two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_observations_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_observations_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_observations_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='observations'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_observations_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_observations_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_observations_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [ ]:
#piControl SSTs - annual mean, djf and jja***

filename= '../outputs/netcdf/sst_ann/PMIP4_piControl_sst_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_ann'
a=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
a

filename= '../outputs/netcdf/sst_djf/PMIP4_piControl_sst_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_djf'
b=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
b

filename= '../outputs/netcdf/sst_jja/PMIP4_piControl_sst_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='sst_spatialmean_jja'
c=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
c

filename= '../outputs/netcdf/sst_ann/PMIP3_piControl_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_piControl_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_piControl_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

#piControl precipitation - annual mean, djf and jja***

filename= '../outputs/netcdf/pr_ann/PMIP4_piControl_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_piControl_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_piControl_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

filename= '../outputs/netcdf/pr_ann/PMIP3_observations_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_piControl_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_piControl_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m

#piControl two metres above surface - annual mean, djf and jja***

filename= '../outputs/netcdf/tas_ann/PMIP4_piControl_tas_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='tas_spatialmean_ann'
n=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
n

filename= '../outputs/netcdf/tas_djf/PMIP4_piControl_tas_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='tas_spatialmean_djf'
o=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
o

filename= '../outputs/netcdf/tas_jja/PMIP4_piControl_tas_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='tas_spatialmean_jja'
p=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
p

filename= '../outputs/netcdf/tas_ann/PMIP3_piControl_tas_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='tas_spatialmean_ann'
q=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
q

filename= '../outputs/netcdf/tas_djf/PMIP3_piControl_tas_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='tas_spatialmean_djf'
r=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
r

filename= '../outputs/netcdf/tas_jja/PMIP3_piControl_tas_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='tas_spatialmean_jja'
s=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
s

In [6]:
#PMIP3 SST and Precipitation ann mean, djf and jja ensemble means

filename= '../outputs/netcdf/sst_ann/PMIP3_piControl_sst_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='sst_spatialmean_ann'
d=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
d

filename= '../outputs/netcdf/sst_djf/PMIP3_piControl_sst_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='sst_spatialmean_djf'
f=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
f

filename= '../outputs/netcdf/sst_jja/PMIP3_piControl_sst_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='sst_spatialmean_jja'
g=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
g

filename= '../outputs/netcdf/pr_ann/PMIP3_observations_pr_spatialmean_ann.nc'
pmip_v='PMIP3'
experiment_name='observations'
variable_name='pr_spatialmean_ann'
k=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
k

filename= '../outputs/netcdf/pr_djf/PMIP3_piControl_pr_spatialmean_djf.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='pr_spatialmean_djf'
l=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
l

filename= '../outputs/netcdf/pr_jja/PMIP3_piControl_pr_spatialmean_jja.nc'
pmip_v='PMIP3'
experiment_name='piControl'
variable_name='pr_spatialmean_jja'
m=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
m


{'ACCESS-ESM1-5': '../data/piControl/ACCESS-ESM1-5_piControl.cvdp_data.101-1000.nc', 'BCC-CSM1-1': '../data/piControl/BCC-CSM1-1_piControl.cvdp_data.1-500.nc', 'CCSM4': '../data/piControl/CCSM4_piControl.cvdp_data.250-1300.nc', 'CESM2': '../data/piControl/CESM2_piControl.cvdp_data.701-1200.nc', 'CNRM-CM5': '../data/piControl/CNRM-CM5_piControl.cvdp_data.1850-2699.nc', 'CNRM-CM6-1': '../data/piControl/CNRM-CM6-1_piControl.cvdp_data.1850-2349.nc', 'COSMOS-ASO': '../data/piControl/COSMOS-ASO_piControl.cvdp_data.0-399.nc', 'CSIRO-Mk3-6-0': '../data/piControl/CSIRO-Mk3-6-0_piControl.cvdp_data.1-500.nc', 'CSIRO-Mk3L-1-2': '../data/piControl/CSIRO-Mk3L-1-2_piControl.cvdp_data.1-1000.nc', 'EC-Earth3-LR': '../data/piControl/EC-Earth3-LR_piControl.cvdp_data.2219-2419.nc', 'FGOALS-f3-L': '../data/piControl/FGOALS-f3-L_piControl.cvdp_data.600-1160.nc', 'FGOALS-g2': '../data/piControl/FGOALS-g2_piControl.cvdp_data.201-900.nc', 'FGOALS-g3': '../data/piControl/FGOALS-g3_piControl.cvdp_data.200-699.nc

['BCC-CSM1-1',
 'CCSM4',
 'CNRM-CM5',
 'COSMOS-ASO',
 'CSIRO-Mk3-6-0',
 'CSIRO-Mk3L-1-2',
 'EC-EARTH-2-2',
 'FGOALS-g2',
 'FGOALS-s2',
 'GISS-E2-1-G',
 'GISS-E2-R',
 'HadCM3',
 'HadGEM2-CC',
 'HadGEM2-ES',
 'IPSL-CM5A-LR',
 'MPI-ESM-P',
 'MRI-CGCM3']

In [10]:
filename= '../outputs/netcdf/pr_ann/PMIP4_piControl_pr_spatialmean_ann.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_ann'
h=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
h

filename= '../outputs/netcdf/pr_djf/PMIP4_piControl_pr_spatialmean_djf.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_djf'
i=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
i

filename= '../outputs/netcdf/pr_jja/PMIP4_piControl_pr_spatialmean_jja.nc'
pmip_v='PMIP4'
experiment_name='piControl'
variable_name='pr_spatialmean_jja'
j=ensemble_mean(pmip_v,variable_name,experiment_name,filename)
j

{'ACCESS-ESM1-5': '../data/piControl/ACCESS-ESM1-5_piControl.cvdp_data.101-1000.nc', 'BCC-CSM1-1': '../data/piControl/BCC-CSM1-1_piControl.cvdp_data.1-500.nc', 'CCSM4': '../data/piControl/CCSM4_piControl.cvdp_data.250-1300.nc', 'CESM2': '../data/piControl/CESM2_piControl.cvdp_data.701-1200.nc', 'CNRM-CM5': '../data/piControl/CNRM-CM5_piControl.cvdp_data.1850-2699.nc', 'CNRM-CM6-1': '../data/piControl/CNRM-CM6-1_piControl.cvdp_data.1850-2349.nc', 'COSMOS-ASO': '../data/piControl/COSMOS-ASO_piControl.cvdp_data.0-399.nc', 'CSIRO-Mk3-6-0': '../data/piControl/CSIRO-Mk3-6-0_piControl.cvdp_data.1-500.nc', 'CSIRO-Mk3L-1-2': '../data/piControl/CSIRO-Mk3L-1-2_piControl.cvdp_data.1-1000.nc', 'EC-EARTH-2-2': '../data/piControl/EC-EARTH-2-2_piControl.cvdp_data.1860-1899.nc', 'EC-Earth3-LR': '../data/piControl/EC-Earth3-LR_piControl.cvdp_data.2219-2419.nc', 'FGOALS-f3-L': '../data/piControl/FGOALS-f3-L_piControl.cvdp_data.600-1160.nc', 'FGOALS-g2': '../data/piControl/FGOALS-g2_piControl.cvdp_data.20

/Users/milesryan-cummings/opt/anaconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'amm_pr_regression_mon' has multiple fill values {-1073741800.0, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/Users/milesryan-cummings/opt/anaconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'amm_tas_regression_mon' has multiple fill values {-1073741800.0, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/Users/milesryan-cummings/opt/anaconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'amo_pr_regression_lowpass_mon' has multiple fill values {-1073741800.0, 1e+20}, decoding all values to NaN.
  decode_timedelta=decode_timedelta,
/Users/milesryan-cummings/opt/anaconda3/envs/xesmf_env/lib/python3.7/site-packages/xarray/conventions.py:500: SerializationWarning: variable 'amo_pr_regre

<xarray.DataArray 'pr_spatialmean_ann' (lat: 180, lon: 360)>
array([[0.20405769, 0.20404174, 0.20402303, ..., 0.20404685, 0.20406533,
        0.20406555],
       [0.183872  , 0.18432228, 0.18484206, ..., 0.18282363, 0.18312951,
        0.18347433],
       [0.0998273 , 0.10222415, 0.10503136, ..., 0.09470557, 0.09602647,
        0.09776114],
       ...,
       [0.56916634, 0.56838835, 0.5680799 , ..., 0.57111498, 0.57019754,
        0.56976946],
       [0.58712381, 0.58665735, 0.58624119, ..., 0.58856449, 0.58808116,
        0.58759846],
       [0.58865905, 0.58850333, 0.58832656, ..., 0.58918478, 0.58903435,
        0.58883607]])
Coordinates:
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Attributes:
    regrid_method:  bilinear
<xarray.DataArray 'pr_spatialmean_ann' (lat: 180, lon: 360)>
array([[0.19604986, 0.19636508, 0.1966454 , ..., 0.1950517 , 0.19537621,
        0.195714

['ACCESS-ESM1-5',
 'CESM2',
 'CNRM-CM6-1',
 'EC-Earth3-LR',
 'FGOALS-f3-L',
 'FGOALS-g3',
 'GISS-E2-1-G',
 'HadGEM3-GC31-LL',
 'INM-CM4-8',
 'IPSL-CM6A-LR',
 'MIROC-ES2L',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NESM3',
 'NorESM1-F',
 'NorESM2-LM',
 'UofT-CCSM-4']